In [4]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor as DTG
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, BaggingRegressor

In [5]:
# real estate data
data = {
    "train_data": "train.csv",
    "test_data": "test.csv"
}

url = "https://raw.githubusercontent.com/atn-iastate/STAT-502-Kaggle/master/data/"

train_data = pd.read_csv("{}{}".format(url, data['train_data']), delimiter=",")
test_data = pd.read_csv("{}{}".format(url, data['test_data']), delimiter=",")

In [74]:
# adding house_age
train_data['yr_sold'] = train_data['date'] // 10000
train_data['yr_sold'].head()
train_data['house_age'] = train_data['yr_sold'] - train_data['yr_built']

# adding house_age
test_data['yr_sold'] = test_data['date'] // 10000
test_data['yr_sold'].head()
test_data['house_age'] = test_data['yr_sold'] - test_data['yr_built']

In [75]:
# plit
train_data['renovated'] = train_data['yr_renovated'].apply(lambda x: 1 if x > 0 else 0)
train_data['zipcode_indicator'] = 1
pivot_train_data = train_data.pivot(columns = 'zipcode', values='zipcode_indicator')
for i in pivot_train_data.columns:
    pivot_train_data[i] = pivot_train_data[i].apply(lambda x: 0 if x != 1 else 1)
    train_data[i] = pivot_train_data[i]

# plit
test_data['renovated'] = test_data['yr_renovated'].apply(lambda x: 1 if x > 0 else 0)
test_data['zipcode_indicator'] = 1
pivot_test_data = test_data.pivot(columns = 'zipcode', values='zipcode_indicator')
for i in pivot_test_data.columns:
    pivot_test_data[i] = pivot_test_data[i].apply(lambda x: 0 if x != 1 else 1)
    test_data[i] = pivot_test_data[i]



In [76]:
train_x = train_data.drop(columns = ['zipcode_indicator', 'property', 'date', 'price', 'yr_built', 
                                     'yr_renovated', 'yr_sold'])
train_y = train_data['price']

x = np.asarray(train_x)
y = np.asarray(train_y)

test_x = test_data.drop(columns = ['id', 'zipcode_indicator', 'property', 'date', 'yr_built', 
                                     'yr_renovated', 'yr_sold'])
test_x = np.asarray(test_x)



In [77]:
# AdaBoostedTree
tree = DTG(min_samples_split = 20, min_samples_leaf = 10).fit(x, y)
y_predict = tree.predict(test_x)
y_predict = np.asarray([max(x, 0) for x in y_predict])

In [78]:
output = pd.DataFrame(test_data['id'])
output['price'] = y_predict

In [79]:
output.to_csv(r'E:\IASTATE\STAT502\kaggle\data\output.csv', index=False)